# Debugging A Simple Image Processing Program Using The Compute Sanitizer
## 07 Memory leaks

In this step, we will focus on the leak-detection capabilities of the compute-sanitizer. Indeed, the memcheck tool has an optional functionality other than
detecting invalid memory accesses. It also allows to detect memory leaks, that
is to say, device memory that has not been released when the program ends. To
enable that check, we need to add the `--leak-check=full` flag. We also need to
call `cudaDeviceReset()` at the end of our program for it work properly, which
is the case here. Let's try to use it on our program:

```sh
    compute-sanitizer --leak-check full ./convolution
```

We can observe that the tool reports leak errors, giving some details about
them.

```
========= COMPUTE-SANITIZER
Image ./checkerboard.png size: 32 * 32
Image ./icon.png size: 32 * 32
Grid size: 2 * 2 * 1
Block size: 31 * 31 * 1
Grid size: 2 * 2 * 1
Block size: 31 * 31 * 1
Image ./coffee.png size: 1936 * 1792
Grid size: 63 * 58 * 1
Block size: 31 * 31 * 1
========= Leaked 13877248 bytes at 0x7efd74000000
=========     Saved host backtrace up to driver entry point at cudaMalloc time
=========     Host Frame: [0x31cf17]
=========                in /usr/lib/x86_64-linux-gnu/libcuda.so.1
=========     Host Frame: [0x3a08e]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x1141b]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x45afc]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xb173]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:main [0xaa37]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__libc_start_main [0x270b3]
=========                in /lib/x86_64-linux-gnu/libc.so.6
=========     Host Frame:_start [0x9c0e]
=========                in /home/nvidia/sanitizer-demo/./convolution
========= 
========= Leaked 4096 bytes at 0x7efd77000000
=========     Saved host backtrace up to driver entry point at cudaMalloc time
=========     Host Frame: [0x31cf17]
=========                in /usr/lib/x86_64-linux-gnu/libcuda.so.1
=========     Host Frame: [0x3a08e]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x1141b]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x45afc]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xb173]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:main [0xa580]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__libc_start_main [0x270b3]
=========                in /lib/x86_64-linux-gnu/libc.so.6
=========     Host Frame:_start [0x9c0e]
=========                in /home/nvidia/sanitizer-demo/./convolution
========= 
========= Leaked 4096 bytes at 0x7efd77001000
=========     Saved host backtrace up to driver entry point at cudaMalloc time
=========     Host Frame: [0x31cf17]
=========                in /usr/lib/x86_64-linux-gnu/libcuda.so.1
=========     Host Frame: [0x3a08e]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x1141b]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x45afc]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xb173]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:main [0xa659]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__libc_start_main [0x270b3]
=========                in /lib/x86_64-linux-gnu/libc.so.6
=========     Host Frame:_start [0x9c0e]
=========                in /home/nvidia/sanitizer-demo/./convolution
========= 
========= Leaked 200 bytes at 0x7efd77002000
=========     Saved host backtrace up to driver entry point at cudaMalloc time
=========     Host Frame: [0x31cf17]
=========                in /usr/lib/x86_64-linux-gnu/libcuda.so.1
=========     Host Frame: [0x3a08e]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x1141b]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x45afc]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xb198]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:main [0xa685]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__libc_start_main [0x270b3]
=========                in /lib/x86_64-linux-gnu/libc.so.6
=========     Host Frame:_start [0x9c0e]
=========                in /home/nvidia/sanitizer-demo/./convolution
========= 
========= LEAK SUMMARY: 13885640 bytes leaked in 4 allocations
========= ERROR SUMMARY: 4 errors
```

Indeed, we forgot to call `cudaFree()` on some of our device buffers
allocated with `cudaMalloc()`. Please try to find said buffers, and free them
when they are not in use anymore. Verify that all the leaks are fixed by
recompiling and running the above command again.

Although in this example, leaking the memory has no visible consequences, in
long running programs, failing to release the memory when it is not needed
anymore leads to unnecessary device memory pressure which can lead
the program to run out of memory when it is avoidable.

[Continue to step 08.](08_unused_memory.ipynb)
